In [ ]:
def scrape()
for type in types:
    scraper = sntwitter.TwitterSearchScraper(f'{game} until:{until_temp} since:{since_temp}')
    for i, tweet in enumerate(scraper.get_items()):
        try:
            data = [
                tweet.date,
                tweet.id,
                tweet.rawContent,
                tweet.user.username,
                tweet.likeCount,
                tweet.retweetCount,
                tweet.lang,
                game
            ]
            if data[4] == data[5]:
                continue
            else:
                data[2] = pre_process(data[2])
                if data[2] == ' ' or data[2] == '':
                    continue
                else:
                    tweets.append(data)
                    if i >= 500:
                        break
        except:
            break

In [8]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
from tqdm.auto import tqdm
from datetime import datetime, timedelta

games = ['CSGO', 'Valorant', 'Overwatch', 'Apex Legends', 'PUBG', 'Fortnite', 'League of Legends', 'Minecraft', 'Roblox', 'GTAV', 'Borderlands2']
types = ['Music', '(plot,story)', 'Gameplay', '(graphics,graphic)', '(characters,character)', 'UI', 'Enviroment', '(Control,controls)', 'community']
now = datetime.now()
date_only = now.date()

until = date_only
since = date_only - timedelta(1)

tweets = []
def pre_process(text):
    # Usuwanie linków
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    # Napraw wbudowane linki
    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)

    # Usuwanie znaków nowej linii
    text = re.sub('[\r\n]+', ' ', text)
    
    # Usuwanie @
    text = re.sub(r'@\w+', '', text)
    
    # Usuwanie #
    text = re.sub(r'#\w+', '', text)

    # Usuń wielokrotności spacji
    text = re.sub('\s+',' ', text)
    
    # Przełóż na lowercase
    text = text.lower()
    return text

# Przeglądamy 500 tweetów dla danego hasztagu
pbar2 = tqdm(total=735)
pbar = tqdm(total=len(games))
for i in range(7,14,7):
    until_temp = until - timedelta(i)
    since_temp = since - timedelta(i)
    pbar.reset()
    for game in games:
        for t in types:
            scraper = sntwitter.TwitterSearchScraper(f'{game} {t} until:{until_temp} since:{since_temp}')
            for i, tweet in enumerate(scraper.get_items()):
                try:
                    data = [
                        tweet.date,
                        tweet.id,
                        tweet.rawContent,
                        tweet.user.username,
                        tweet.likeCount,
                        tweet.retweetCount,
                        tweet.lang,
                        game,
                        t
                    ]
                    if data[4] == data[5]:
                        continue
                    else:
                        data[2] = pre_process(data[2])
                        if data[2] == ' ' or data[2] == '':
                            continue
                        else:
                            tweets.append(data)
                            if i >= 500:
                                break
                except:
                    break

        scraper = sntwitter.TwitterSearchScraper(f'{game} until:{until_temp} since:{since_temp} -{types}')
        for i, tweet in enumerate(scraper.get_items()):
            try:
                data = [
                    tweet.date,
                    tweet.id,
                    tweet.rawContent,
                    tweet.user.username,
                    tweet.likeCount,
                    tweet.retweetCount,
                    tweet.lang,
                    game,
                    "other"
                ]
                if data[4] == data[5]:
                    continue
                else:
                    data[2] = pre_process(data[2])
                    if data[2] == ' ' or data[2] == '':
                        continue
                    else:
                        tweets.append(data)
                        if i >= 500:
                            break
            except:
                break
        pbar.update(1)
    pbar2.update(2)

a=pd.DataFrame(tweets, columns=['date', 'id', 'content', 'username','like_count','retweet_count', 'language', 'game','type'])

a.head()



  0%|          | 0/735 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

,date,id,content,username,like_count,retweet_count,language,game,type
0,2023-04-05 21:41:41+00:00,1643730653735981057,"back on the grind! warming up on csgo, later w...",prozeq,3,1,en,CSGO,Music
1,2023-04-05 21:05:35+00:00,1643721567594074114,buying albums irl when you can buy csgo music...,MrBuildAPC,1,0,en,CSGO,Music
2,2023-04-05 20:35:15+00:00,1643713933579264001,friends : why do you play csgo? me : soundtrip...,karlishgg,6,0,en,CSGO,Music
3,2023-04-05 17:40:26+00:00,1643669941584027649,バービーの映画にこの曲流れないのマジンガーz???? aqua - barbie girl ...,mameloff_csgo,0,1,ja,CSGO,Music
4,2023-04-05 16:44:17+00:00,1643655811946905600,i need both and to have music kits in csgo asap,_angelslash,2,0,en,CSGO,Music


In [9]:
a['id'] = a['id'].astype('string')
a['date'] = a['date'].dt.tz_localize(None)
a.to_excel(r'export_dataframe.xlsx', index=False)